#Extract
Authors: Gustavo FLEURY && Induraj RAMAMURTHY

Project: https://github.com/gustavofleury/Audit_Reports_NLP

In [0]:
!pip install tika

  Created wheel for tika: filename=tika-1.19-cp36-none-any.whl size=29224 sha256=dd12392719c8bfeccf640ac9c47c17172aa8f0e269f9052febf59c760502bc44
  Stored in directory: /root/.cache/pip/wheels/b4/db/8a/3a3f0c0725448eaa92703e3dda71e29dc13a119ff6c1036848
Successfully built tika


In [0]:
#Libraries
from tika import parser
import pandas as pd
import unicodedata as ud
import re
from pathlib import Path

from time import gmtime, strftime
from os import listdir
from os.path import isfile, join


In [0]:
#Take Files From GoogleDrive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Verify Files needed to PARSE
df_file_STR_Report="/content/drive/My Drive/Research/DataSets/DF_Reports_STR.pkl"
dfSTR=pd.read_pickle(df_file_STR_Report)
filesDid=dfSTR['FileName'].tolist()

mypath='/content/drive/My Drive/Reports/' #Indu Download Directory
# mypath='/content/drive/My Drive/Research/DataSets/Reports/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
lenNotParsed= len(onlyfiles) - len(list(set(onlyfiles) & set(filesDid)))                                     
    
print("N Files: " + str(len(onlyfiles)) + " - N Parsed: " + str(len(filesDid)) + " N Need Parse:" + str(lenNotParsed))

N Files: 10884 - N Parsed: 10879 N Need Parse:5


In [0]:
# len(list(set(onlyfiles) & set(filesDid)))
# set(onlyfiles)
# len(set(filesDid))
# onlyfiles
# lstDidNotinNew=[item for item in filesDid if item not in list(set(onlyfiles) & set(filesDid)) ]
lstDidNotinOLD=[item for item in onlyfiles if item not in list(set(onlyfiles) & set(filesDid)) ]

print(len(lstDidNotinNew))
print(len(lstDidNotinOLD))
print(len(list(set(onlyfiles) & set(filesDid))))


NameError: ignored

In [0]:
#Function to do the Parse in Blocks 
# (Avoid to lose all job if some error occur)

filesErrors=[]
def parsePDF( numberReports ):
  #Open FILE
  df_file_STR_Report="/content/drive/My Drive/Research/DataSets/DF_Reports_STR.pkl"
  dfSTR=pd.read_pickle(df_file_STR_Report)
  filesDid=dfSTR['FileName'].tolist()

  #LOOP FILES TIKA PARSE
  lst=[]
  columns=['FileName','Content','MetaData']
  mypath='/content/drive/My Drive/Reports/' #Indu Download Directory
  # mypath='/content/drive/My Drive/Research/DataSets/Reports/'
  onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
  lstNotParsed=[item for item in onlyfiles if item not in list(set(onlyfiles) & set(filesDid)) ]
  for fileName in lstNotParsed[0: numberReports ]:
    if fileName in filesDid:
      fileName=fileName
      print(".", end='')
      # print("Already did: " + fileName)
    else:
      raw = parser.from_file('/content/drive/My Drive/Reports/'+fileName)
      if raw['content'] != None:
        lst.append( [fileName, raw['content'], raw['metadata']])
        print("Parse: " + fileName + " Len Report Content: " + str(len(raw['content'])) )
      else:
        filesErrors.append(fileName)
        print("ERROR: " + fileName) 
  #
  dfSTR_tmp = pd.DataFrame(lst, columns=columns)
  dfSTRNew=pd.concat( [dfSTR, dfSTR_tmp], ignore_index=True, sort=False )

  #Save result
  #dfSTR.to_pickle("/content/drive/My Drive/Research/DataSets/DF_Reports_STR_"+strftime("%Y.%m.%d.%Hh%Mm%S", gmtime())+"OLD.pkl")
  dfSTRNew.to_pickle(df_file_STR_Report)
  print(str(numberReports) + " Finalizado!")

In [0]:
#CALL PDFParse Function in BLOCKS
mypath='/content/drive/My Drive/Reports/' #Indu Download Directory
# mypath='/content/drive/My Drive/Research/DataSets/Reports/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
for x in range(100,len(onlyfiles)+100,100):
  parsePDF(x)

ERROR: 6895.pdf
ERROR: 7158.pdf
ERROR: 7297.pdf
ERROR: 11862.pdf
ERROR: 6051.pdf
100 Finalizado!


KeyboardInterrupt: ignored

In [0]:
# onlyfiles[210:215]
# raw = parser.from_file('/content/drive/My Drive/Research/DataSets/Reports/11862.pdf')
# raw
print(filesErrors) 

['11862.pdf', '11862.pdf', '11862.pdf', '11862.pdf', '11862.pdf', '11862.pdf', '11862.pdf', '11862.pdf', '11862.pdf', '11862.pdf', '11862.pdf', '11862.pdf', '11862.pdf']


In [0]:
# #TEST for Ond Report
# raw = parser.from_file('/content/drive/My Drive/Research/DataSets/Reports/6443.pdf')
# report=raw['content']
# reportOriginal=report

# #CREATE 1st Versiokn of DF_Reports_STR.pkl
# fileName='6443.pdf'
# lst=[]
# lst.append( [fileName, raw['content'], raw['metadata']])
# columns=['FileName','Content','MetaData']
# dfSTR = pd.DataFrame(lst, columns=columns)
# df_file_STR_Report=Path("/content/drive/My Drive/Research/DataSets/DF_Reports_STR.pkl")
# dfSTR.to_pickle(df_file_STR_Report)

In [0]:
  df_file_STR_Report="/content/drive/My Drive/Research/DataSets/DF_Reports_STR.pkl"
  dfSTR=pd.read_pickle(df_file_STR_Report)

In [0]:
#Import the list of OrgSubordinados.
df=pd.read_pickle("/content/drive/My Drive/Research/DataSets/OrgaoSubordinado_Unique.pkl")

In [0]:
#TEST the list of Subordinade Orgs;
#df.head()
#df.count()
# df[ df['Nome Órgão Subordinado'].str.contains(pat="correios") ]

#Verify if the 
subNameToFind="correios"
# subNameToFind="secretaria"
for i in df['Nome Órgão Subordinado']:
  if (i.decode("utf-8").find(subNameToFind)) != -1 :
    print(i.decode("utf-8"))

In [0]:
#Normalize, delete excess /N and spaces, Lowercase,
report=ud.normalize('NFKD', report).encode('ASCII','ignore').decode('ASCII').strip().lower()
report=report.replace('\n', '')
report=re.sub(' +', ' ', report)
report[0:1000]

'dinheiro publico e da sua conta www.portaldatransparencia.gov.br 1 unidade auditada: empresa brasileira de correios e telegrafos exercicio: 2012 processo: 53101.000288/2012-92 municipio: brasilia - df relatorio no: 201306248 uci executora: sfc/drcom - coordenacao-geral de auditoria da area de comunicacoes _______________________________________________ analise gerencial senhor coordenador-geral, em atendimento a determinacao contida na ordem de servico n.o 201306248, e consoante o estabelecido na secao iii, capitulo vii da instrucao normativa sfc n.o 01, de 6.4.2001, apresentamos os resultados dos exames realizados sobre a prestacao de contas anual apresentada pela empresa brasileira de correios e telegrafos ect. 1. introducao os trabalhos de campo conclusivos foram realizados no periodo de 3.6.2013 a 30.8.2009, por meio de testes, analises e consolidacao de informacoes coletadas ao longo do exercicio sob exame e a partir da apresentacao do processo de contas pela unidade auditada, em

In [0]:
# Terms/Phrases to Delete
list2Delete=['dinheiro publico e da sua conta',
             'www.portaldatransparencia.gov.br',
             'presidencia da republica', 
             'controladoria-geral da uniao',
             'secretaria federal de controle interno'
             ]
for text in list2Delete:
  report=report.replace(text,'')

report=re.sub(' +', ' ', report)
report=re.sub('________________', '_', report)

report[0:1000]

' 1 unidade auditada: empresa brasileira de correios e telegrafos exercicio: 2012 processo: 53101.000288/_2012-92 municipio: brasilia - df relatorio no: 201306248 uci executora: sfc/_drcom - coordenacao-geral de auditoria da area de comunicacoes _____ analise gerencial senhor coordenador-geral, em atendimento a determinacao contida na ordem de servico n.o 201306248, e consoante o estabelecido na secao iii, capitulo vii da instrucao normativa sfc n.o 01, de 6.4.2001, apresentamos os resultados dos exames realizados sobre a prestacao de contas anual apresentada pela empresa brasileira de correios e telegrafos ect. 1. introducao os trabalhos de campo conclusivos foram realizados no periodo de 3.6.2013 a 30.8.2009, por meio de testes, analises e consolidacao de informacoes coletadas ao longo do exercicio sob exame e a partir da apresentacao do processo de contas pela unidade auditada, em estrita observancia as normas de auditoria aplicaveis ao servico publico federal. nenhuma restricao foi

In [0]:
#Type of Report
#Obtain from the complet list DF of Links
fileNumber='6443'

dfLinks=pd.read_pickle("/content/drive/My Drive/Research/DataSets/DF_Links_Reports.pkl")
dfLinksClean=dfLinks.drop_duplicates()
reportType =  dfLinksClean['Linha de Atuação'][ dfLinksClean['Links'].str.contains("/"+fileNumber+".pdf") ].to_string()
print(reportType)


56    Auditoria Anual de Contas


In [0]:
#SEPARATE the HEADER of REPORT
reportHEADER=re.findall('unidade\sauditada:\s(.*?)resultados\sdos\strabalhos',report)[0]
print(reportHEADER)

empresa brasileira de correios e telegrafos exercicio: 2012 processo: 53101.000288/_2012-92 municipio: brasilia - df relatorio no: 201306248 uci executora: sfc/_drcom - coordenacao-geral de auditoria da area de comunicacoes _____ analise gerencial senhor coordenador-geral, em atendimento a determinacao contida na ordem de servico n.o 201306248, e consoante o estabelecido na secao iii, capitulo vii da instrucao normativa sfc n.o 01, de 6.4.2001, apresentamos os resultados dos exames realizados sobre a prestacao de contas anual apresentada pela empresa brasileira de correios e telegrafos ect. 1. introducao os trabalhos de campo conclusivos foram realizados no periodo de 3.6.2013 a 30.8.2009, por meio de testes, analises e consolidacao de informacoes coletadas ao longo do exercicio sob exame e a partir da apresentacao do processo de contas pela unidade auditada, em estrita observancia as normas de auditoria aplicaveis ao servico publico federal. nenhuma restricao foi imposta a realizacao 

In [0]:
#FIND ORG_SUB of ReportHEADER
#Collect 1st the GivenName
orgaoSub=re.findall('(.*?)\sexercicio:',reportHEADER)[0]
orgaoSubCod=''

#Collect the OrgaoSubCod (IF IT HAS)
for index, row in df.iterrows():
  orgaoSubTest=(row['Nome Órgão Subordinado']).decode("utf-8")
  #print(orgaoSub)
  if reportHEADER.find(orgaoSubTest) != -1:
    orgaoSub=orgaoSubTest
    orgaoSubCod=row['Código Órgão Subordinado']

print(str(orgaoSub) + ' - ' + str(orgaoSubCod) )

empresa brasileira de correios e telegrafos - 


In [0]:
#FIND Report Number
#years='(2014|2015|2016|2017|2018|2019)'
reportNum=re.findall('relatorio\sno:\s(\d+)', reportHEADER)[0]
print(reportNum)

201306248


In [0]:
#FIND DATE of Analysis
month = "(janeiro|fevereiro|marco|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro)"
dateBEGIN=re.findall('de\s(\d{2})\sde\s'+month+'\sa\s', reportHEADER)
dateEND=re.findall('a\s(\d{2})\sde\s'+month+'\sde\s(\d{4})', reportHEADER)
print(dateBEGIN)
print(dateEND)
reportYear=dateEND[0][2]
print(reportYear)
monthDict={'janeiro':'1','fevereiro':'2','marco':'3','abril':'4','maio':'5','junho':'6','julho':'7','agosto':'8','setembro':'9','outubro':'10','novembro':'11','dezembro':'12'}
reportMesBegin=monthDict[dateBEGIN[0][1]]
reportMesEnd=monthDict[dateEND[0][1]]
print(reportMesBegin)
reportBegin=reportYear+'/'+reportMesBegin
reportEnd=reportYear+'/'+reportMesEnd

#This type of Report (AAC) is for all year
reportYear=re.findall('exercicio:\s(\d+)', reportHEADER)[0]
reportBegin=reportYear+'/1'
reportEnd=reportYear+'/12'
print(reportBegin)
print(reportEnd)

2012/1
2012/12


In [0]:
# FIND TITLE CONSTATACAO
constatType="constatacao|informacao"
constat=re.findall('(\d\.\d\.\d\.\d+)\s('+constatType+')(.*?)fato(.*?)\d\.\d\.\d\.\d+\s'+constatType, report)
print(constat)

[('', '', '', ''), ('', '', '', ''), ('', '', '', ''), ('', '', '', ''), ('', '', '', ''), ('', '', '', ''), ('1.1.1.1', 'informacao', ' informacao geral sobre o escopo da auditoria anual de contas ', ' em atendimento a determinacao contida na ordem de servico n.o 201306248, e consoante o estabelecido na secao iii, capitulo vii da instrucao normativa sfc n.o 01, de 6.4.2001, apresentamos os resultados dos exames realizados sobre a prestacao de contas anual apresentada pela empresa brasileira de correios e telegrafos ect. os trabalhos de campo foram realizados por meio de testes, analises e consolidacao de informacoes coletadas ao longo do exercicio sob exame e a partir da apresentacao do processo de contas pela unidade auditada, em estrita observancia as normas de auditoria aplicaveis ao servico publico federal. alem disso, no intuito de avaliar a gestao da unidade examinada em 2012, foram utilizadas, como subsidio da analise ora apresentada, as informacoes contidas nos seguintes relat

In [0]:
#Mount TABLE
#ReportNumber|
organizeLst=[]
for numC, typeC, titleC, textC in constat:
  if len(numC) > 0:
    labelTypeC = 1 if typeC=='constatacao' else 0
    organizeLst.append([orgaoSubCod,orgaoSub,reportNum,reportType,reportBegin,reportEnd,numC,typeC, titleC, textC, labelTypeC])
print(organizeLst)
columns=['OrgSubCod','OrgSub','ReportNumber','ReportType','DateBegin','DateEnd','ConstNumber','ConstTYPE','ConstTitle','ConstText','LABEL_ConstType']
organizedDF = pd.DataFrame(organizeLst, columns=columns)

[['', 'empresa brasileira de correios e telegrafos', '201306248', '56    Auditoria Anual de Contas', '2012/1', '2012/12', '1.1.1.1', 'informacao', ' informacao geral sobre o escopo da auditoria anual de contas ', ' em atendimento a determinacao contida na ordem de servico n.o 201306248, e consoante o estabelecido na secao iii, capitulo vii da instrucao normativa sfc n.o 01, de 6.4.2001, apresentamos os resultados dos exames realizados sobre a prestacao de contas anual apresentada pela empresa brasileira de correios e telegrafos ect. os trabalhos de campo foram realizados por meio de testes, analises e consolidacao de informacoes coletadas ao longo do exercicio sob exame e a partir da apresentacao do processo de contas pela unidade auditada, em estrita observancia as normas de auditoria aplicaveis ao servico publico federal. alem disso, no intuito de avaliar a gestao da unidade examinada em 2012, foram utilizadas, como subsidio da analise ora apresentada, as informacoes contidas nos seg

In [0]:
organizedDF

,OrgSubCod,OrgSub,ReportNumber,ReportType,DateBegin,DateEnd,ConstNumber,ConstTYPE,ConstTitle,ConstText,LABEL_ConstType
0,,empresa brasileira de correios e telegrafos,201306248,56 Auditoria Anual de Contas,2012/1,2012/12,1.1.1.1,informacao,informacao geral sobre o escopo da auditoria ...,em atendimento a determinacao contida na orde...,0
1,,empresa brasileira de correios e telegrafos,201306248,56 Auditoria Anual de Contas,2012/1,2012/12,1.1.1.3,informacao,bens adquiridos e nao distribuidos e bens dis...,o item 2.8 do relatorio 2013-004-gefc-orcamen...,0
2,,empresa brasileira de correios e telegrafos,201306248,56 Auditoria Anual de Contas,2012/1,2012/12,1.1.1.5,constatacao,inconformidade nos repasses dos valores mensa...,"conforme informando na constatacao 11, relata...",1
3,,empresa brasileira de correios e telegrafos,201306248,56 Auditoria Anual de Contas,2012/1,2012/12,1.1.2.1,constatacao,falhas nos registros do encaminhamento de enc...,em conformidade com o plano anual de atividad...,1
4,,empresa brasileira de correios e telegrafos,201306248,56 Auditoria Anual de Contas,2012/1,2012/12,1.1.2.3,informacao,falha na apuracao do indicador trafego de obj...,em conformidade com o plano anual de atividad...,0
5,,empresa brasileira de correios e telegrafos,201306248,56 Auditoria Anual de Contas,2012/1,2012/12,1.1.2.5,informacao,informacao sobre os ajustes realizados nas re...,em conformidade com o plano anual de atividad...,0
6,,empresa brasileira de correios e telegrafos,201306248,56 Auditoria Anual de Contas,2012/1,2012/12,1.1.3.3,informacao,existencia de comite gestor de tecnologia da ...,"foi observado, atraves da documentacao analis...",0
7,,empresa brasileira de correios e telegrafos,201306248,56 Auditoria Anual de Contas,2012/1,2012/12,1.1.3.7,constatacao,inobservancia da metodologia de planejamento ...,"em analise ao processo licitatorio 12000019, ...",1
8,,empresa brasileira de correios e telegrafos,201306248,56 Auditoria Anual de Contas,2012/1,2012/12,2.1.1.3,constatacao,ausencia de recomendacao nos relatorios de au...,"o memorando no 01994/_2012-audit, de 3.10.201...",1
9,,empresa brasileira de correios e telegrafos,201306248,56 Auditoria Anual de Contas,2012/1,2012/12,1.4.1.1,constatacao,8 falha grave nao ha. ra2013-002-glic licitac...,"s; tomada de acoes junto ao contratado, se nec...",1


In [0]:
#Print DF to HTML
organizedDF.to_html('organizedDF.html')


OSError: ignored